In [1]:
import numpy as np #For array managment
import math
import random
import sys
sys.path.append('..')
from Simulation import simulation as sim

pygame 2.1.0 (SDL 2.0.16, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def DKL(l1,l2):
    ###
    #DKL between two arrays, they have to be pmfs
    #We only integrate over the support of
    return(np.sum([l1[i]*math.log(l1[i]/l2[i]) for i in range(len(l1)) if l2[i]!=0]))

In [3]:
S = 5 #Amount of sources
xDim = 5 #The dimension of our relative state space

#declared by the user range(3-10) from 3x3 to 10x10
grid_dimension = 4
fields_number = grid_dimension*grid_dimension
grid = []
state_memory = []

In [4]:
def findWall(x):
    ###Check if the agent is near a wall on square x
    # Returns a list of ints, one for each wall (in Python 1<=>True, 0<=>False)
    res = [0, 0, 0, 0]
    if x % grid_dimension == 0:  # If the agent is near the left-side wall, update the list
        res[0] = 1
    if x % grid_dimension == (grid_dimension - 1):  # If the agent is near the right-side wall, update the list
        res[2] = 1
    if x // grid_dimension == 0:  # If the agent is near the bottom-side wall, update the list
        res[3] = 1
    if x // grid_dimension == (grid_dimension - 1):  # If the agent is near the top-side wall, update the list
        res[1] = 1

    return (res)  # Return the list

In [5]:
def target_direction():
    # arrange the list in the correct order
    startindex = grid_dimension
    stopindex = 0
    step = -1
    field_number = 0
    cnt = 0
    for i in range(startindex, stopindex, step):
        for j in range(0, grid_dimension, 1):
            field_number = (grid_dimension * i) - grid_dimension + j
            grid.insert(cnt, field_number)
            cnt += 1

    ## Fields for whose GO right determined on the basis of the diagonal
    squaresGoRight = []
    rows = grid_dimension  # const value
    cols = grid_dimension
    counter = 0

    for j in range(fields_number):
        if (counter == rows):  # in 1st row take all, in next take 1 less
            counter = 0
            cols = cols - 1  # decrise
        if (counter < cols):
            squaresGoRight.append(grid[j])
        counter = counter + 1

    ## Fields for whose GO up, rest of the grid
    squaresGoUp = [squareNumber for squareNumber in grid if squareNumber not in squaresGoRight]
    return squaresGoRight, squaresGoUp

In [6]:
def return_target(x):
    ###Return the pmf corresponding to the target for square x (the behavior is picked depending on the square)
    squaresGoRight, squaresGoUp = target_direction()

    if x in squaresGoRight:  # For these squares, we want to go right
        return ([0.01, 0.01, 0.01, 0.96, 0.01])  # Return the pmf corresponding to moving right

    if x in squaresGoUp:  # For these squares, we want to go up
        return ([0.01, 0.01, 0.96, 0.01, 0.01])  # Return the pmf corresponding to moving top

    return ([0.96, 0.01, 0.01, 0.01, 0.01])  # Else, we want to stay on position

In [7]:
def rand_obstacle_location():
    # rand obstacle location
    # get diffent amount of obstacle depended on grid size, for 3x3 1 obstacle, 4x4 2obstacle etc.

    if (grid_dimension == 3):
        number_of_obtacles = 1
    else:
        number_of_obtacles = (grid_dimension - 2)

    field_number_with_obtacle = []

    for i in range(number_of_obtacles):
        result = random.randrange(1, (fields_number - 1))  # without 1st and last square (for 5x5) w/o 0 and 24
        if i > 0:
            while result+grid_dimension-1 in field_number_with_obtacle or result-grid_dimension+1 in field_number_with_obtacle:
                result = random.randrange(1, (fields_number - 1))
        field_number_with_obtacle.append(result)

    print(f"Fields with obstacles: {field_number_with_obtacle}")
    return field_number_with_obtacle

In [8]:
def return_reward(x):
    ###Return the reward function (voded as a list of its values) for square x
    # Square zero (the starting point) has a reward of -1, and square 4 (the middle) has a reward of -100
    # Again, this function picks the reward based on the square where the agent is
    output = []
    if x == (fields_number-1):
        output = [0, -100, -100, -100, -100]  # end point stay in place
    elif (x + grid_dimension) in field_number_with_obtacle and not (x + 1) in field_number_with_obtacle:
        output = [-1, -1, -1000, 0, -1]  # obstacle ABOVE agent
    elif (x + 1) in field_number_with_obtacle and not (x + grid_dimension) in field_number_with_obtacle:
        output = [-1, -1, 0, -1000, -1]  # obstacle RIGHT side agent
    elif (x - 1) in field_number_with_obtacle:
        output = [0, -1000, 0, 0, 0]  # obstacle LEFT agent
    elif (x - grid_dimension) in field_number_with_obtacle:
        output = [0, 0, 0, 0, -1000]  # obstacle BELOW agent
    elif x in field_number_with_obtacle:
        output = [-1000, 0, 0, 0, 0]  # ARRR! agent in the obstacle
    elif (x + grid_dimension) in field_number_with_obtacle and (x + 1) in field_number_with_obtacle:
        output = [-1, 0, -1000, -1000, -1]  # obstacle ABOVE and RIGHT side of agent
    else:
        output = [-100, 0, 0, 0, 0]  # else
    return output

In [9]:
def return_sources(x):
    ###Return the behavior of the sources for square x
    source_left = [0.01, 0.96, 0.01, 0.01, 0.01]  # By default, the four sources propose the corresponding movements (left, top, right, bottom)
    source_top = [0.01, 0.01, 0.96, 0.01, 0.01]
    source_right = [0.01, 0.01, 0.01, 0.96, 0.01]
    source_bot = [0.01, 0.01, 0.01, 0.01, 0.96]
    source_stay = [0.96, 0.01, 0.01, 0.01, 0.01]

    if findWall(x)[0]:  # If we are near a wall (or more), the corresponding movement is replaced by "stay in place"
        source_left = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[1]:
        source_top = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[2]:
        source_right = [0.96, 0.01, 0.01, 0.01, 0.01]
    if findWall(x)[3]:
        source_bot = [0.96, 0.01, 0.01, 0.01, 0.01]

    return (np.array([source_stay, source_left, source_top, source_right, source_bot, source_stay]))

In [10]:
def available_States(x):
    free_grids = findWall(x)  # check if there's no walls
    available_actions = [i for i, x in enumerate(free_grids) if x == 0]
    for iter in range(len(available_actions)):
        available_actions[iter] += 1
    if available_actions[0] != 0:
        available_actions.insert(0,0)

    return available_actions

In [11]:
def new_state(position, action):
    ###
    # On the other hand, this function takes into account an actual state (x) and an action OR a relative state from that actual state (action) and gives the resulting actual state
    if action == 0:
        return(position)
    if action == 1:
        return(position - 1)
    if action == 2:
        return(position + grid_dimension)
    if action == 3:
        return(position + 1)
    if action == 4:
        return(position - grid_dimension)

In [12]:
def receding_horizon_crowdsourcing(tHor, xStart):
    ###
    #Main loop, calculate the weights through a receding horizon and choose a policy
    #The idea is that we restric the state space to the relative states available to us and use crowdsourcing on this new state space
    timeHor = list(range(tHor))
    timeHor.reverse()
    rHat = [0]*xDim #Reward modifier, same dimension as the relative state space
    weights = np.zeros((xDim, S)) #We have S weights per relative state
    if tHor == 0: #If the time horizon is zero (greedy case), we have to explicitly tell python to still do a loop
        timeHor = [0]
    for k in timeHor:
        for x in available_States(xStart): #In the restricted state space
            rHat[x] = -min(weights[x]) #Calulate \hat{r}
            r = np.add(return_reward(new_state(xStart,x)),rHat) #Modify the reward array with \hat{r} by an elementwise multiplication
            sources = return_sources(new_state(xStart,x)) #Get the sources and the target
            target = return_target(new_state(xStart,x))
            for i in range(S): #Main loop
                a = DKL(sources[i],target) - np.dot(sources[i],r) #Calculate the weight and store it
                weights[x][i] = a
        if k==0: #If we're at the last iteration
            j = np.argmin(weights[0]) #Get the minimum weight
            return(return_sources(xStart)[j]) #Return the corresponding policy

In [13]:
def loop(xStart):
    ###
    #Simulation loop
    pmf = receding_horizon_crowdsourcing(2, xStart)
    action = np.argmax(pmf) #We use a maximum sampling
    x = new_state(xStart, action) #Get the corresponding new state
    print('PMF: ' + str(pmf))
    print('New state: ' + str(x))
    return(x)

In [14]:
#main
x = 0
state_memory.append(x)
field_number_with_obtacle = rand_obstacle_location()
# field_number_with_obtacle = [6,9]
for i in range(10): #let's take 10 loops
    # if(x != fields_number-1):
    x = loop(x)
    state_memory.append(x)

sim(grid_dimension, field_number_with_obtacle, state_memory)

Fields with obstacles: [10, 14]
PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 1
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 5
PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 6
PMF: [0.01 0.01 0.01 0.96 0.01]
New state: 7
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 11
PMF: [0.01 0.01 0.96 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15
PMF: [0.96 0.01 0.01 0.01 0.01]
New state: 15


SystemExit: 

D:\Aplikacje\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
